# Part 3. Model Training & Evaluation - biLSTM

## Prepare embedding matrix and metadata

In [13]:
import json
from pathlib import Path

import numpy as np

embedding_path = Path("models/embedding_matrix_oov.npy")
index_from_word_path = Path("models/index_from_word_oov.json")

embedding_matrix = np.load(embedding_path)
with index_from_word_path.open() as f:
    index_from_word = json.load(f)

## Prepare dataset

In [14]:
from utils.text import tokenize
from datasets import load_dataset

dataset = load_dataset("rotten_tomatoes")
train_dataset = tokenize(dataset["train"])
val_dataset = tokenize(dataset["validation"])
test_dataset = tokenize(dataset["test"])

In [15]:
from utils.text import token_to_index

train_dataset = token_to_index(dataset=train_dataset, index_from_word=index_from_word)
val_dataset = token_to_index(dataset=val_dataset, index_from_word=index_from_word)
test_dataset = token_to_index(dataset=test_dataset, index_from_word=index_from_word)

In [16]:
train_dataset = train_dataset.select_columns(["label", "original_len", "indexes"])
val_dataset = val_dataset.select_columns(["label", "original_len", "indexes"])
test_dataset = test_dataset.select_columns(["label", "original_len", "indexes"])

In [17]:
train_dataset.set_format(type="torch")
val_dataset.set_format(type="torch")
test_dataset.set_format(type="torch")

In [6]:
train_dataset

Dataset({
    features: ['label', 'original_len', 'indexes'],
    num_rows: 8530
})

## Train RNN - biLSTM Model

Hieuristic search with Optuna

In [ ]:
import optuna
from utils.train import train_rnn_model_with_parameters

_N_TRIALS = 150
SEARCH_SPACE = {
    "batch_size": [2048],
    "learning_rate": [1e-1, 1e-2, 1e-3, 1e-4],
    "optimizer_name": ["Adam", "Adagrad",],
    # biLSTM Model Parameters
    "hidden_dim": [256, 128, 64, 32],
    "num_layers": [1, 2, 4],
    "sentence_representation_type": ["last", "average", "max"],
}

def objective(trial):
    hidden_dim = trial.suggest_categorical("hidden_dim", SEARCH_SPACE["hidden_dim"])
    num_layers = trial.suggest_int("num_layers", min(SEARCH_SPACE["num_layers"]), max(SEARCH_SPACE["num_layers"]))
    optimizer_name = trial.suggest_categorical("optimizer_name", SEARCH_SPACE["optimizer_name"])
    batch_size = trial.suggest_categorical("batch_size", SEARCH_SPACE["batch_size"])
    learning_rate = trial.suggest_categorical("learning_rate", SEARCH_SPACE["learning_rate"])
    sentence_representation_type = trial.suggest_categorical("sentence_representation_type", SEARCH_SPACE["sentence_representation_type"])
    
    log_message = f"---------- batch_size_{batch_size}; lr_{learning_rate}; optimizer_{optimizer_name}; hidden_dim_{hidden_dim}; num_layers_{num_layers}; sentence_representation_{sentence_representation_type} ----------"
    print(log_message)

    val_acc = train_rnn_model_with_parameters(
        embedding_matrix=embedding_matrix,
        train_dataset=train_dataset,
        val_dataset=val_dataset,
        batch_size=batch_size,
        learning_rate=learning_rate,
        optimizer_name=optimizer_name,
        hidden_dim=hidden_dim,
        num_layers=num_layers,
        sentence_representation_type=sentence_representation_type,
        show_progress=True,
        log_dir="bilstm",
        rnn_type="LSTM",
        bidirectional=True,
        freeze_embedding=False
    )
    
    return val_acc

# Set up the Optuna study
study = optuna.create_study(direction="maximize") 
study.optimize(objective, n_trials=_N_TRIALS)

best_params = study.best_params

## Train Results Analysis

In [12]:
from utils.analytics import load_tensorboard_logs

train_results_df = load_tensorboard_logs(log_dir="tb_logs/bilstm")
train_results_df = train_results_df.sort_values(
    by=["val_acc"], ascending=False
).reset_index(drop=True)

train_results_df.head(20)

,val_acc,batch_size,hidden_dim,learning_rate,optimizer_name,train_loss,train_acc,num_layers,sentence_representation_type,freeze,epoch,val_loss,filename
0,0.791745,2048,128,0.0100,Adagrad,0.092932,0.961358,1,average,False,8.0,0.524783,events.out.tfevents.1731018611.Bernices-MacBoo...
1,0.787054,2048,256,0.0010,Adam,0.296868,0.886810,2,max,False,6.0,0.497451,events.out.tfevents.1731128945.yuriarch.3871.21
2,0.786116,2048,64,0.0010,Adam,0.115555,0.967680,1,max,False,10.0,0.485980,events.out.tfevents.1731128170.yuriarch.3871.15
3,0.783302,2048,64,0.0010,Adam,0.218479,0.933186,2,max,False,9.0,0.475823,events.out.tfevents.1731128386.yuriarch.3871.18
4,0.781426,2048,256,0.0010,Adam,0.176542,0.952663,1,max,False,7.0,0.485084,events.out.tfevents.1731128704.yuriarch.3871.20
5,0.779550,2048,64,0.0010,Adagrad,0.316925,0.881039,1,max,False,21.0,0.487491,events.out.tfevents.1731017839.Bernices-MacBoo...
6,0.779550,2048,64,0.0010,RMSprop,0.163863,0.952972,2,max,False,5.0,0.479183,events.out.tfevents.1731128265.yuriarch.3871.16
7,0.778612,2048,256,0.0010,Adagrad,0.298538,0.867635,1,max,False,14.0,0.489787,events.out.tfevents.1730998423.Bernices-MacBoo...
8,0.778612,2048,64,0.0010,Adam,0.200404,0.953757,2,average,False,8.0,0.537270,events.out.tfevents.1731130675.yuriarch.3871.30
9,0.777674,2048,256,0.0001,Adam,0.395819,0.847594,2,max,False,18.0,0.496814,events.out.tfevents.1731136356.yuriarch.3871.44


Configuration of the best model

In [18]:
best_rnn_model_configuration = train_results_df.head(1)
best_rnn_model_configuration

,val_acc,batch_size,hidden_dim,learning_rate,optimizer_name,train_loss,train_acc,num_layers,sentence_representation_type,freeze,epoch,val_loss,filename
0,0.791745,2048,128,0.01,Adagrad,0.092932,0.961358,1,average,False,8.0,0.524783,events.out.tfevents.1731018611.Bernices-MacBoo...


## Performance on Test Dataset

In [ ]:
from utils.analytics import test_top_n_models
from models.RNN import RNNClassifier

test_results_df = test_top_n_models(train_results_df, RNNClassifier, test_dataset, n=10)

In [20]:
test_results_df

,test_acc,test_loss,val_acc,batch_size,hidden_dim,learning_rate,optimizer_name,train_loss,train_acc,num_layers,sentence_representation_type,freeze,epoch,val_loss,filename
0,0.783302,0.638366,0.791745,2048,128,0.0100,Adagrad,0.092932,0.961358,1,average,False,8.0,0.524783,events.out.tfevents.1731018611.Bernices-MacBoo...
1,0.788931,0.467992,0.787054,2048,256,0.0010,Adam,0.296868,0.886810,2,max,False,6.0,0.497451,events.out.tfevents.1731128945.yuriarch.3871.21
2,0.803002,0.43742,0.786116,2048,64,0.0010,Adam,0.115555,0.967680,1,max,False,10.0,0.485980,events.out.tfevents.1731128170.yuriarch.3871.15
3,0.798311,0.442497,0.783302,2048,64,0.0010,Adam,0.218479,0.933186,2,max,False,9.0,0.475823,events.out.tfevents.1731128386.yuriarch.3871.18
4,0.769231,0.456733,0.781426,2048,256,0.0010,Adam,0.176542,0.952663,1,max,False,7.0,0.485084,events.out.tfevents.1731128704.yuriarch.3871.20
5,0.795497,0.46093,0.779550,2048,64,0.0010,Adagrad,0.316925,0.881039,1,max,False,21.0,0.487491,events.out.tfevents.1731017839.Bernices-MacBoo...
6,0.797373,0.444567,0.779550,2048,64,0.0010,RMSprop,0.163863,0.952972,2,max,False,5.0,0.479183,events.out.tfevents.1731128265.yuriarch.3871.16
7,0.796435,0.452854,0.778612,2048,256,0.0010,Adagrad,0.298538,0.867635,1,max,False,14.0,0.489787,events.out.tfevents.1730998423.Bernices-MacBoo...
8,0.783302,0.586049,0.778612,2048,64,0.0010,Adam,0.200404,0.953757,2,average,False,8.0,0.537270,events.out.tfevents.1731130675.yuriarch.3871.30
9,0.78424,0.4787,0.777674,2048,256,0.0001,Adam,0.395819,0.847594,2,max,False,18.0,0.496814,events.out.tfevents.1731136356.yuriarch.3871.44
